In [2]:
import numpy as np
import pandas as pd
import os
from pathlib import Path

from IPython.display import HTML
from tqdm import tqdm

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [12]:
out_dir_path = '../output/1/{}'

In [ ]:
vals = []

for game_id in tqdm(sorted(os.listdir(out_dir_path.format('')))):
    for play_id in tqdm(sorted(os.listdir(out_dir_path.format(f'{game_id}')))):
        for frame_id in sorted(os.listdir(out_dir_path.format(f'{game_id}/{play_id}'))):
            print(out_dir_path.format(f'{game_id}/{play_id}/{frame_id}'))
            with open(out_dir_path.format(f'{game_id}/{play_id}/{frame_id}'), 'rb') as f:
                all = np.load(f, allow_pickle=True)
                print(list(all.keys()))
                eppa_ind = all['eppa_ind']
                player_info = all['ind_info']
                print(player_info)
                print(2/)
                eppa = np.sum(eppa_ind, axis=-1, where=player_info[:, 1]=='OFF')
                vals.append((game_id, play_id, frame_id, eppa))

len(vals)

In [17]:
eppa_sums = list(map(lambda x: x[3].sum(), vals))

In [26]:
a = vals[0]
print(a)
print(a[3].shape)

('2018090600', '1037', '11.npz', array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]))
(6600, 40)


In [18]:
df = pd.DataFrame(list(map(lambda x: (x[0], x[1], int(x[2][:-4]),x[3].sum()), vals)), columns=['gameId', 'playId', 'frameId', 'eppa_sum'])
df.head(10)

,gameId,playId,frameId,eppa_sum
0,2018090600,1037,11,-0.279331
1,2018090600,1037,12,-0.290279
2,2018090600,1037,13,-0.285995
3,2018090600,1037,14,-0.272747
4,2018090600,1037,15,-0.224963
5,2018090600,1037,16,-0.132762
6,2018090600,1037,17,-0.004429
7,2018090600,1037,18,0.137314
8,2018090600,1037,19,0.192732
9,2018090600,1037,20,0.292319


In [35]:
df[['eppa_sum']].describe()

,eppa_sum
count,2714.000000
mean,0.630291
std,0.321244
min,0.078087
25%,0.391682
50%,0.589892
75%,0.844659
max,1.669557


In [46]:
df.sort_values('eppa_sum', ascending=False)

,gameId,playId,frameId,eppa_sum
2446,2018093005,2703,24,1.669557
2447,2018093005,2703,25,1.660921
2442,2018093005,2703,20,1.660528
2441,2018093005,2703,19,1.658250
2443,2018093005,2703,21,1.657809
...,...,...,...,...
607,2018091001,3904,13,0.094118
606,2018091001,3904,12,0.089882
605,2018091001,3904,11,0.087769
2295,2018093002,1058,12,0.082920


In [66]:
df = pd.read_csv('./results/eppa_epa_yards.csv').drop(columns={'Unnamed: 0'})
df.head()

,gameId,drive_play_id_started,count,drive_result,total_yards,total_epa,total_eppa,passer_name
0,2018090600,37,4,DOWNS,10,-1.655583,-0.078233,M.Ryan
1,2018090600,299,2,PUNT,4,-1.536305,0.456902,N.Foles
2,2018090600,402,5,FIELD_GOAL,29,0.259991,0.503018,M.Ryan
3,2018090600,677,1,PUNT,6,0.895417,-0.192395,N.Foles
4,2018090600,889,2,PUNT,-7,-2.463996,0.048063,M.Ryan


In [67]:
df['total_yards'] = df.groupby(['passer_name', 'gameId'], as_index=False).total_yards.transform('sum')
df['total_epa'] = df.groupby(['passer_name', 'gameId'], as_index=False).total_epa.transform('sum')
df['total_eppa'] = df.groupby(['passer_name', 'gameId'], as_index=False).total_eppa.transform('sum')
df['games'] = df.groupby(['passer_name'], as_index=False).gameId.transform('nunique')
# df = df.loc[df.total_yards>10]

In [75]:
tots = df.groupby(['passer_name', 'gameId'], as_index=False)[['passer_name', 'gameId', 'games', 'total_yards', 'total_epa', 'total_eppa']].head(1).loc[df.games>12]
tots.groupby(['passer_name', ])

array([16, 14, 15, 13])

In [69]:
import numpy as np

In [70]:
tots.sort_values(['passer_name', 'gameId']).total_yards.autocorr()

0.25298936883935397

In [71]:
tots.sort_values(['passer_name', 'gameId']).total_epa.autocorr()

0.15308730201437518

In [72]:
tots.sort_values(['passer_name', 'gameId']).total_eppa.autocorr()

0.020599420370688033

In [65]:
totY.loc[totY.passer_name=='M.Ryan'].sort_values(['passer_name', 'gameId']).total_yards.autocorr()

-0.07302671749260502

In [33]:
totY.sample(frac=1).total_yards.autocorr()

-0.032567570393464376